<a href="https://colab.research.google.com/github/laloflogar/RB2026/blob/main/FRED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import pandas as pd
import numpy as np

def descarga_fred_series(series_dict, fechainicio, fechafin):
    # Token de acceso a la API de FRED
    api_key = 'b24da1f1e67d32a43455a8d8ba7d4d53'
    base_url = 'https://api.stlouisfed.org/fred/series/observations'

    all_data = []

    for serie, nombre in series_dict.items():
        # Parámetros de la solicitud
        params = {
            'series_id': serie,
            'api_key': api_key,
            'file_type': 'json',
            'observation_start': fechainicio,
            'observation_end': fechafin
        }

        response = requests.get(base_url, params=params)

        if response.status_code != 200:
            print(f'Error en la consulta para la serie {nombre}, código {response.status_code}')
            continue

        raw_data = response.json()

        # FRED devuelve los datos en la llave 'observations'
        if 'observations' in raw_data and len(raw_data['observations']) > 0:
            data = raw_data['observations']
            df = pd.DataFrame(data)

            # Limpieza de datos: FRED usa "." para valores faltantes
            df['value'] = df['value'].replace('.', np.nan)
            df['value'] = df['value'].astype(float)

            # Formateo de fechas
            df['date'] = pd.to_datetime(df['date'])
            df.set_index('date', inplace=True)

            # Seleccionar solo la columna de valor y renombrarla
            df = df[['value']].rename(columns={'value': nombre})
            all_data.append(df)
        else:
            print(f"No se encontraron datos para la serie {nombre}")

    if all_data:
        # Unimos todas las series por su fecha (índice)
        df_final = pd.concat(all_data, axis=1)
        return df_final
    else:
        print("No se descargaron datos.")
        return None

# --- Ejemplo de uso ---

# Series de ejemplo: GDP (PIB EE.UU.) y UNRATE (Tasa de desempleo)
series_fred = {
    'GDP': 'PIB USA (Nominal)',
    'UNRATE': 'Tasa de Desempleo USA',
    'PAYEMS': 'Nómina no agrícola'
}

fechainicio = '2010-01-01'
fechafin = pd.Timestamp.today().strftime('%Y-%m-%d')

base = descarga_fred_series(series_fred, fechainicio, fechafin)

if base is not None:
    print("Datos de FRED descargados exitosamente ✅ :")
    print(base.tail())

Datos de FRED descargados exitosamente ✅ :
            PIB USA (Nominal)  Tasa de Desempleo USA  Nómina no agrícola
date                                                                    
2025-07-01          31095.089                    4.2            159511.0
2025-08-01                NaN                    4.3            159485.0
2025-09-01                NaN                    4.4            159593.0
2025-10-01                NaN                    NaN            159488.0
2025-11-01                NaN                    4.6            159552.0


In [19]:
import plotly.express as px

# Filter the DataFrame to include only the 'Nómina no agrícola' column
df_nomina = base[['Nómina no agrícola']].copy()

# Create the Plotly figure
fig = px.line(
    df_nomina,
    x=df_nomina.index,
    y='Nómina no agrícola',
    title='<b>Gráfica 00: Nómina No Agrícola de EE.UU.</b><br><sup>Serie mensual de empleos no agrícolas</sup>'
)

# Update layout for desired styling
fig.update_layout(
    plot_bgcolor='white', # Set plot background to white
    paper_bgcolor='white', # Set paper background to white
    width=1000,  # Set width for square shape
    height=700,
    xaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=False,
        title_text='' # Remove x-axis title
    ),
    yaxis=dict(
        title_text=' ',
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=False,
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=1,
        title_font=dict(color="black"),
        tickfont=dict(color="black")
    ),
    showlegend=True, # Ensure legend is visible
    title_x=0.5, # Center the title

annotations=[
        # Footnote
        dict(
            xref="paper", yref="paper",
            x=0, y=-0.11, # Position below the x-axis
            xanchor="left", yanchor="top",
            text="Fuente: Elaboración propia con datos de FRED.",
            font=dict(size=12),
            showarrow=False
        )
    ],
legend=dict(
        x=0.94, # Right side of the plot
        y=0.01, # Bottom of the plot
        xanchor='right', # Anchor the legend to the right
        yanchor='bottom', # Anchor the legend to the bottom
        bgcolor='rgba(255, 255, 255, 0.5)', # Optional: translucent background
        bordercolor='rgba(0, 0, 0, 0)'
    )

),


# Show the plot
fig.show()